In [ ]:
from transformers import AutoModelForCausalLM, AutoModelForSeq2SeqLM
from transformers import AutoTokenizer

import json
import math
import numpy as np
import torch
import torch.nn.functional as F

In [ ]:
def calculate_choice_probability(model, tokenizer, input_text, alternativa):
    input_text = input_text.strip()

    initial_inputs = tokenizer(input_text, return_tensors="pt").input_ids[0]
    model_inputs = tokenizer(input_text + " " + alternativa, return_tensors="pt")
    complete_inputs = model_inputs.input_ids[0]

    position_ids = model_inputs.attention_mask.cumsum(-1) - 1
    position_ids.masked_fill_(model_inputs.attention_mask == 0, 1)

    s = 0
    with torch.no_grad():
        out = model(
            model_inputs.input_ids,
            attention_mask=model_inputs.attention_mask,
            position_ids=position_ids,
        )

        logits_resposta = out.logits[0, len(initial_inputs) - 1 : -1]
        tokenized_alternativa = complete_inputs[len(initial_inputs) :]

        for l, expected_token in zip(logits_resposta, tokenized_alternativa):
            probs = F.softmax(l)
            s += math.log(probs[expected_token])

    return s / len(tokenized_alternativa)

In [ ]:
def predict_dataset(data, ignore_files=None):
    if not ignore_files:
        ignore_files = []

    certo, errado = 0, 0

    for k, v in data.items():
        print(k)
        if k in ignore_files:
            continue

        prob_answers = []

        for a in v["answers"]:
            context = v["context"]
            question = v["question"]

            input_text = f"{context}\n{question}"

            prob_answers.append(calculate_choice_probability(model, tokenizer, input_text, a))

        result = np.array(prob_answers)
        correct = np.argmax(result) == v["correct_answer"]
        if correct:
            certo += 1
        else:
            errado += 1

        with open(f"/gdrive/MyDrive/datasets/mcqa/enem_challenge/outputs/{k}.json", "w") as f:
            json.dump({"k": k, "result": [float(r) for r in result], "certo": str(correct)}, f)

    return {
        "acc": certo / (certo + errado),
        "certo": certo,
        "errado": errado
    }

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
# @title Modelo

model_name = 'maritaca-ai/sabia-7b'  # @param {type: "string"}

In [ ]:
DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/700 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.


In [ ]:
import glob
current_results = []
for file in glob.glob("/gdrive/MyDrive/datasets/mcqa/enem_challenge/outputs/*.json"):
    current_results.append(file.split('.')[0].split('/')[-1])

In [ ]:
datasets = ["ENEMdataset_with_ds.json", "ENEMdataset_without_ds.json"]
datasets_path = "/gdrive/MyDrive/datasets/mcqa/enem_challenge"


results = {}
for dataset_name in datasets:
    with open(datasets_path + "/" + dataset_name, "r") as f:
        data = json.load(f)

    results[dataset_name] = predict_dataset(data, ignore_files=current_results)
    print(results[dataset_name])


2012_97
2012_98
2012_100
2012_101
2012_102
2012_103
2012_104
2012_105
2012_107
2012_108
2012_109
2012_110
2012_111
2012_113
2012_115
2012_117
2012_118
2012_119
2012_120
2012_121
2012_122
2012_124
2012_125
2012_126
2012_127
2012_128
2012_129
2012_130
2012_134
2012_135
2016_96
2016_97
2016_98
2016_99
2016_100
2016_102
2016_103
2016_104
2016_105
2016_106
2016_107
2016_108
2016_110
2016_111
2016_112
2016_113
2016_114
2016_115
2016_116
2016_117
2016_119
2016_120
2016_121
2016_122
2016_124
2016_125
2016_126
2016_127
2016_128
2016_129
2016_130
2016_131
2016_132
2016_135
2016_133
2016_134
2016_03
2016_01
2016_02
2016_04
2016_05
2016_07
2016_08
2016_10
2016_11
2016_12
2016_13
2016_16
2016_17
2016_15
2016_18
2016_19
2016_20
2016_22
2016_23
2016_25
2016_26
2016_27
2016_29
2016_30
2016_31
2016_33
2016_36
2016_34
2016_35
2016_39
2016_38
2016_42
2016_43
2016_44
2016_45
2016_46
2016_47
2016_51
2016_54
2016_55
2016_56
2016_60
2016_61
2016_62
2016_63
2016_64
2016_67
2016_68
2016_69
2016_72
2016_73
2016

We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)
<ipython-input-2-0f08f2657d78>:23: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  probs = F.softmax(l)


2012_58
2012_63
2012_65
2012_68
2012_71
2012_74
2012_75
2012_76
2012_79
2012_80
2012_81
2012_82
2012_84
2012_87
2012_88
2009_92
2009_95
2009_98
2009_99
2009_102
2009_103
2009_104
2009_106
2009_107
2009_108
2009_110
2009_111
2009_113
2009_115
2009_116
2009_117
2009_118
2009_119
2009_120
2009_121
2009_122
2009_123
2009_124
2009_128
2009_129
2009_130
2009_131
2009_132
2009_133
2009_134
2009_135
2009_01
2009_03
2009_04
2009_05
2009_06
2009_07
2009_08
2009_09
2009_10
2009_11
2009_13
2009_23
2009_27
2009_31
2009_33
2009_34
2009_37
2009_39
2009_40
2009_41
2009_42
2009_46
2009_47
2009_48
2009_49
2009_50
2009_51
2009_52


KeyboardInterrupt: 

In [ ]:
with open(datasets_path + "/results.json", "w") as f:
    json.dump(results, f)

TESTE:

esperado: `[-4.59502864142497]`

In [ ]:
data = {
    "question": "Ao analisarem a correlação entre os hábitos e o perfil socioeconômico dos usuários da internet no Brasil, os pesquisadores",
    "id": "ENEM_2022_41",
    "options": [
        "apontam o desenvolvimento econômico como solução para ampliar o uso da rede.",
        "questionam a crença de que o acesso à informação é igualitário e democrático.",
        "afirmam que o uso comercial da rede é a causa da exclusão de minorias.",
        "refutam o vínculo entre níveis de escolaridade e dificuldade de acesso.",
        "condicionam a expansão da rede à elaboração de políticas inclusivas."
    ],
    "label": "b",
    "context": "São vários os fatores, internos e externos, que influenciam os hábitos das pessoas no acesso à internet, assim como nas práticas culturais realizadas na rede. A utilização das tecnologias de informação e comunicação está diretamente relacionada aos aspectos como: conhecimento de seu uso, acesso à linguagem letrada, nível de instrução, escolaridade, letramento digital etc. Os que detêm tais recursos (os mais escolarizados) são os que mais acessam a rede e também os que possuem maior índice de acumulatividade das práticas. A análise dos dados nos possibilita dizer que a falta de acesso à rede repete as mesmas adversidades e exclusões já verificadas na sociedade brasileira no que se refere a analfabetos, menos escolarizados, negros, população indígena e desempregados. Isso significa dizer que a internet, se não produz diretamente a exclusão, certamente a reproduz, tendo em vista que os que mais a acessam são justamente os mais jovens, escolarizados, remunerados, trabalhadores qualificados, homens e brancos. SILVA, F. A. B.; ZIVIANE, P.; GHEZZI, D. R. As tecnologias digitais e seus usos."
}

contexto = data["context"]
pergunta = data["question"]

alternativas_ex = ["apontam o desenvolvimento econômico como solução."]

model = AutoModelForCausalLM.from_pretrained("pierreguillou/gpt2-small-portuguese").to(DEVICE)
tokenizer = AutoTokenizer.from_pretrained("pierreguillou/gpt2-small-portuguese")

In [ ]:
saida_teste = calculate_choice_probability(model, tokenizer, formatar_texto_entrada(contexto, pergunta), alternativas_ex)
print(saida_teste)